In [3]:
import pandas as pd
from datetime import datetime, timezone
from sqlalchemy import text
from GeneradordeReportes.utils.db import get_engine
from MainDatabase.main_database_mapping import field_schema, tabla_por_campo

EXCEL_PATH = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\Main Database\Contract Database - All Regions.xlsx"
SHEET_NAME = 'Master Data Results'
SCHEMA = 'masterdatabase'

engine = get_engine()

def asegurar_tablas_y_columnas():
    with engine.begin() as conn:
        for tabla, campos in tabla_por_campo.items():
            # 1. Verificar si la tabla existe
            result = conn.execute(text(f"""
                SELECT COUNT(*) FROM information_schema.tables
                WHERE table_schema = :schema AND table_name = :tabla
            """), {"schema": SCHEMA, "tabla": tabla})
            tabla_existe = result.scalar() > 0

            if not tabla_existe:
                print(f"🆕 Creando tabla {tabla}")
                columnas_sql = []
                for col in sorted(campos):
                    tipo = next(
                        (sql_type for _, sql_type, tablas in field_schema.values()
                         if _ == col and tablas and tabla in tablas),
                        'TEXT'
                    )
                    tipo = 'INTEGER' if tipo.upper() == 'SMALLINT' else tipo
                    columnas_sql.append(f"{col} {tipo}")
                columnas_sql.append("loaded_at TIMESTAMPTZ")
                conn.execute(text(f"""
                    CREATE TABLE {SCHEMA}.{tabla} (
                        {', '.join(columnas_sql)}
                    );
                """))
            else:
                print(f"✅ Tabla {tabla} ya existe, asegurando columnas…")
                # 2. Verificar y crear columnas que falten
                result = conn.execute(text(f"""
                    SELECT column_name FROM information_schema.columns
                    WHERE table_schema = :schema AND table_name = :tabla
                """), {"schema": SCHEMA, "tabla": tabla})
                columnas_existentes = {r[0] for r in result}

                for col in sorted(campos | {'loaded_at'}):
                    if col not in columnas_existentes:
                        tipo = next(
                            (sql_type for _, sql_type, tablas in field_schema.values()
                             if _ == col and tablas and tabla in tablas),
                            'TEXT'
                        )
                        tipo = 'INTEGER' if tipo.upper() == 'SMALLINT' else tipo
                        print(f"  ➕ Añadiendo columna {col} ({tipo}) a {tabla}")
                        conn.execute(text(f"""
                            ALTER TABLE {SCHEMA}.{tabla}
                            ADD COLUMN {col} {tipo};
                        """))
    print("🎯 Tablas y columnas aseguradas.")

if __name__ == "__main__":
    asegurar_tablas_y_columnas()


💻 Conectado a la base de datos helloworldtree
🆕 Creando tabla contract_tree_information
✅ Tabla contract_farmer_information ya existe, asegurando columnas…
  ➕ Añadiendo columna address (TEXT) a contract_farmer_information
  ➕ Añadiendo columna contract_name (TEXT) a contract_farmer_information
  ➕ Añadiendo columna email (TEXT) a contract_farmer_information
  ➕ Añadiendo columna farmer_number (TEXT) a contract_farmer_information
  ➕ Añadiendo columna loaded_at (TEXT) a contract_farmer_information
  ➕ Añadiendo columna phone (TEXT) a contract_farmer_information
  ➕ Añadiendo columna representative (TEXT) a contract_farmer_information
  ➕ Añadiendo columna shipping_address (TEXT) a contract_farmer_information
  ➕ Añadiendo columna status (TEXT) a contract_farmer_information
✅ Tabla contract_allocation ya existe, asegurando columnas…
  ➕ Añadiendo columna canada_2017_allocation_pct (DECIMAL) a contract_allocation
  ➕ Añadiendo columna canada_2017_trees (INTEGER) a contract_allocation
  ➕

In [5]:
import pandas as pd
from datetime import datetime, timezone
from sqlalchemy import create_engine
from MainDatabase.main_database_mapping import field_schema, tabla_por_campo
from GeneradordeReportes.utils.db import get_engine

# Ruta al archivo Excel
EXCEL_PATH = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\Main Database\Contract Database - All Regions.xlsx"
SHEET_NAME = 'Master Data Results'
SCHEMA = 'masterdatabase'

# Crear conexión
engine = get_engine()

# Leer Excel y preparar columnas
df_raw = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME, dtype={'Contract Code': str})
df_raw.columns = df_raw.columns.str.replace(r"\s+", " ", regex=True).str.strip()

# Renombrar columnas visibles a internas
col_map = {k.strip(): v[0] for k, v in field_schema.items()}
df_raw = df_raw.rename(columns=col_map)
df_raw['loaded_at'] = datetime.now(timezone.utc)

# Forzar conversión numérica en campos con tipos definidos
for visible, (internal, sql_type, _) in field_schema.items():
    if internal in df_raw.columns and sql_type.upper() in ('SMALLINT', 'INTEGER', 'DECIMAL', 'DOUBLE PRECISION'):
        df_raw[internal] = pd.to_numeric(df_raw[internal], errors='coerce')

df_raw['loaded_at'] = datetime.now(timezone.utc)


# Importar por tabla
for tabla, campos in tabla_por_campo.items():
    columnas_finales = ['contract_code'] + sorted([c for c in campos if c in df_raw.columns and c != 'contract_code'])
    if not columnas_finales or 'contract_code' not in columnas_finales:
        print(f"⚠️  No hay columnas válidas para {tabla}, se omite.")
        continue

    df_sub = df_raw[columnas_finales + ['loaded_at']].copy()
    df_sub = df_sub[df_sub['contract_code'].notna()]

    if df_sub.empty:
        print(f"⚠️  No hay registros para {tabla}, se omite.")
        continue

    # Insertar en PostgreSQL
    df_sub.to_sql(
        name=tabla,
        schema=SCHEMA,
        con=engine,
        if_exists='append',
        index=False,
        method='multi'
    )
    print(f"✅ {len(df_sub)} registros insertados en {tabla}")


💻 Conectado a la base de datos helloworldtree
✅ 423 registros insertados en contract_tree_information
✅ 423 registros insertados en contract_farmer_information
✅ 423 registros insertados en contract_allocation
✅ 423 registros insertados en contract_replacements
